# loading data from MySQL
In this notebook you'll find an example of reading a database table (mysql) and writing it to iguazio key value database.
This is useful for importing data to iguazio from external database

In [ ]:
!pip install pymysql 
!pip install v3io-frames

Set the database connection. in this example we are using a public mysql database called Rfam (https://rfam.readthedocs.io/en/latest/database.html)<br>
Then run the sql and load the result into a dataframe

In [ ]:
import os
import pymysql
import pandas as pd

host = os.getenv('mysql-rfam-public.ebi.ac.uk')
port = os.getenv('4497')
user = os.getenv('rfamro')
password = os.getenv('')
database = os.getenv('Rfam ')

conn = pymysql.connect(
    host="mysql-rfam-public.ebi.ac.uk",
    port=int(4497),
    user="rfamro",
    passwd=password,
    db="Rfam",
    charset='utf8mb4')

df = pd.read_sql_query("select rfam_acc,rfam_id,auto_wiki,description,author,seed_source  FROM family",
    conn) 


#df.tail(10)
#df.describe()
#df.info()



## Ingesting data into iguazio database using frames
We'll create a table named "Family" under the examples directory using an iguazio library called frames.
Frames is a library for reading and writing data from and to iguazio. it is used for accessing any type of data including streaming, time-series, key value and files from a single interface

In [ ]:
import pandas as pd
import v3io_frames as v3f
client = v3f.Client('http://v3io-framesd:8080')
tablename = 'examples/family'

In [ ]:
out = client.write('kv', tablename, df)

## Reading from the table
Now that the table is created we'll run a select statement to view the data

In [ ]:
# run only once (load SQL magic)
%load_ext sql
%config SqlMagic.autocommit=False

In [ ]:
%sql select * from v3io.bigdata."/examples/family"

## Remove Data
For cleaning up just run the following

In [ ]:
client.delete('kv',table=tablename)